In [ ]:
te = @__DIR__

sr = joinpath(dirname(te), "src")

_get_prefix(na) = splitext(na)[1]

display(
    symdiff(
        (_get_prefix(na) for na in readdir(sr) if endswith(na, r"\.jl$") && na != "BioLab.jl"),
        (
            _get_prefix(na) for
            na in readdir(te) if endswith(na, r"\.ipynb$") && na != "runtests.ipynb"
        ),
    ),
)

In [ ]:
using BioLab

## TE

In [ ]:
# TODO: @test
display(BioLab.TE)

## CA_

In [ ]:
# TODO: @test
display(BioLab.CA_)

## RA

In [ ]:
# TODO: @test
display(BioLab.RA)

## 🎬

In [ ]:
BioLab.IPYNB.run(@__DIR__, ("$pr.ipynb" for pr in ("runtests",)))